In [5]:
import math
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F


class PositionWiseFFN(nn.Module):
    def __init__(self,ffn_num_iuput,ffn_num_hiddens,ffm_num_outputs,**kwargs):
        super().__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_iuput,ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens,ffm_num_outputs)
        
    def forward(self,X):
        return self.dense2(self.relu(self.dense1(X)))
        
    


d:\anaconda\envs\paddleocr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ffn = PositionWiseFFN(4,4,8)
ffn.eval()
ffn(torch.ones((2,3,4)))


tensor([[[ 0.4363, -0.9838, -0.6535,  0.4282, -0.8677,  0.5715, -0.2057,
          -1.1354],
         [ 0.4363, -0.9838, -0.6535,  0.4282, -0.8677,  0.5715, -0.2057,
          -1.1354],
         [ 0.4363, -0.9838, -0.6535,  0.4282, -0.8677,  0.5715, -0.2057,
          -1.1354]],

        [[ 0.4363, -0.9838, -0.6535,  0.4282, -0.8677,  0.5715, -0.2057,
          -1.1354],
         [ 0.4363, -0.9838, -0.6535,  0.4282, -0.8677,  0.5715, -0.2057,
          -1.1354],
         [ 0.4363, -0.9838, -0.6535,  0.4282, -0.8677,  0.5715, -0.2057,
          -1.1354]]], grad_fn=<ViewBackward0>)

In [6]:
class AddNorm(nn.Module):
    def __init__(self,normalized_shape,dropout,**kwargs):
        super().__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
    
    def forward(self,X,Y):
        return self.ln(self.dropout(Y)+X)
    
# add_norm = AddNorm([3,4],0.5)
# add_norm.eval() 
# add_norm(torch.ones((2,3,4)),torch.ones((2,3,4)))
    
  

In [ ]:
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

In [3]:
class DotProductAttention(nn.Module):
    """缩放点积注意力"""
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)

NameError: name 'nn' is not defined

In [1]:
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])

def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

class MultiHeadAttention(nn.Module):
    def __init__(self,key_size,query_size,value_size,num_hiddens,num_heads,dropout,bias=False,**kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = nn.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size,num_hiddens,bias=bias)
        self.w_k = nn.Linear(key_size,num_hiddens,bias=bias)
        self.w_v = nn.Linear(value_size,num_hiddens,bias=bias)
        self.w_o = nn.Linear(num_hiddens,num_hiddens,bias=bias)
        
    def forward(self,queries,keys,values,valid_lens):
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        
        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)
        
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)
 
 
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()   

NameError: name 'nn' is not defined

In [ ]:
class Encoderlock(nn.Module):
    "transformer编码块"
    def __init__(self,key_size,query_size,value_size,num_hiddens,norm_shape,
                 ffn_num_input,ffn_num_hiddens,num_heads,dropout,**kwargs):
        super().__init__(**kwargs)
        self.attention = 